In [1]:
import sys

sys.path.append('./helpers_models/')
sys.path.append('./data_visualization_and_augmentations/')
sys.path.append('../torch_videovision/')
sys.path.append('./important_csvs/')

from helpers_resnet import *
from focal_loss_2 import *
from load_data_and_augmentations import *
from imbalanced_sampler_3 import MultilabelBalancedRandomSampler
from warp_lsep import *

In [2]:
tensor_transform = get_tensor_transform('ImageNet', False)
train_spat_transform = get_spatial_transform(0)
train_temp_transform = get_temporal_transform()
valid_spat_transform = get_spatial_transform(0)
valid_temp_transform = va.TemporalFit(size=16)

root_dir = '/media/scratch/astamoulakatos/centre_Ch2/'
df = pd.read_csv('./important_csvs/more_balanced_dataset/big_stratified_new.csv')

####################################################
bs = 20
seq_length = 1
df_train = get_df(df, 50, True, False, False)
class_image_paths, end_idx, idx_label = get_indices(df_train, root_dir)
indices, labels = get_final_indices_2d_train(idx_label, end_idx, skip_frames=15, set_step=4, seq_length=1)
indices = torch.cat(indices)
indices = indices[torch.randperm(len(indices))]
labels = []
for i in class_image_paths:
    labels.append(i[2])
labels = np.array(labels)
train_sampler = MultilabelBalancedRandomSampler(
    labels, indices, class_choice="least_sampled"
)
dataset = MyDataset(
        image_paths = class_image_paths,
        seq_length = seq_length,
        temp_transform = train_temp_transform,
        spat_transform = train_spat_transform,
        tensor_transform = tensor_transform,
        length = len(train_sampler),
        lstm = False,
        oned = True,
        augment = True,
        multi = 1)
train_loader = DataLoader(
        dataset,
        batch_size = bs,
        sampler = train_sampler,
        drop_last = True,
        num_workers = 0)
##########################################################################################

#train_loader = get_loader(16, 64, end_idx, class_image_paths, train_temp_transform, train_spat_transform, tensor_transform, False, False)
df_valid = get_df(df, 50, False, True, False)
class_image_paths, end_idx, idx_label= get_indices(df_valid, root_dir)
indices, labels = get_final_indices_2d_valid(idx_label, end_idx, skip_frames=15, set_step=4, seq_length=1)
valid_loader, valid_dataset = get_loader_new(seq_length, bs, indices, class_image_paths, valid_temp_transform,
                                             valid_spat_transform, tensor_transform, False, True, True, 1)

df_test = get_df(df, 50, False, False, True)
class_image_paths, end_idx, idx_label = get_indices(df_test, root_dir)
indices, labels = get_final_indices_2d_valid(idx_label, end_idx, skip_frames=15, set_step=4, seq_length=1)
test_loader, test_dataset = get_loader_new(seq_length, bs, indices, class_image_paths, valid_temp_transform, 
                                            valid_spat_transform, tensor_transform, False, True, True, 1)

In [3]:
#show_batch(train_loader,20)

In [4]:
def imshow(inp, title=None):
    inp = inp.cpu().numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize=(30,30))
    plt.imshow(inp)
    if title is not None:
        plt.title(title, fontsize=30)
    plt.pause(0.001)  # pause a bit so that plots are updated 

In [17]:
def imsave(inp, title=None):
    inp = inp.cpu().numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imsave('./saved-style-images/'+title+'.png',inp)
    #return inp
    #cv2.imwrite('./saved-style-images/'+title+'.png',inp)
    

In [169]:
imgs, labels = next(iter(train_loader))

In [171]:
for l in labels:
    print(l)#labels[4]

tensor([1., 0., 0., 0., 0.])
tensor([1., 0., 1., 0., 0.])
tensor([1., 0., 0., 1., 0.])
tensor([0., 1., 0., 0., 0.])
tensor([1., 0., 0., 0., 1.])
tensor([0., 1., 0., 0., 0.])
tensor([1., 0., 0., 0., 0.])
tensor([1., 0., 1., 0., 0.])
tensor([1., 0., 0., 1., 0.])
tensor([1., 0., 0., 0., 1.])
tensor([0., 1., 0., 0., 0.])
tensor([1., 0., 0., 1., 0.])
tensor([1., 0., 1., 0., 0.])
tensor([1., 0., 0., 0., 1.])
tensor([1., 0., 0., 0., 0.])
tensor([0., 1., 0., 0., 0.])
tensor([1., 0., 1., 0., 0.])
tensor([1., 0., 0., 1., 0.])
tensor([1., 0., 0., 0., 1.])
tensor([1., 0., 0., 0., 0.])


In [176]:
imsave(imgs[13],'exp3')